In [1]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [2]:
import json 
from core.services.base_api import BaseAPIService
from core.loggers import configure_logging
from core.utils.common_utils import join_paths
from sqlalchemy.orm import Session
from fastapi import Depends
from core.databases.db import get_db


In [3]:
logger = configure_logging(__name__)


class ShopeeAPIService(BaseAPIService):
    def __init__(self):
        super().__init__()
        self.shopee_api_url = "https://gappapi.deliverynow.vn/"
        self.city_id = 219 # Da Nang

    @property
    def headers(self):
        return {
            'X-Foody-Access-Token': '',
            'X-Foody-Api-Version': '1',
            'X-Foody-App-Type': '1004',
            'X-Foody-Client-Id': '',
            'X-Foody-Client-Language': 'vi',
            'X-Foody-Client-Type': '1',
            'X-Foody-Client-Version': '3.0.0',
            'Content-Type': 'application/json'
        }

    def get_ids(self):
        response = self.post(
            endpoint=join_paths(self.shopee_api_url, "api/promotion/get_ids"),
            headers=self.headers,
            json={
                "city_id": self.city_id,
                "foody_service_id": 1,
                "promotion_status": 1,
                "sort_type": 0
            }
        )
        ids = response['reply']['promotion_ids']
        return ids

    def get_restaurant_infos(self):
        promotion_ids = self.get_ids()
        response = self.post(
            endpoint=join_paths(self.shopee_api_url, "api/promotion/get_infos"),
            headers=self.headers,
            json={
                "promotion_ids": promotion_ids,
            }
        )
        data = response['reply']['promotion_infos']
        return data
    

In [4]:
# service = ShopeeAPIService()
# restaurant_info =  service.get_restaurant_infos()

# with open('data.json', 'w') as f:
#     f.write(json.dumps(restaurant_info))

In [5]:
with open('data.json', 'r') as f:
    restaurant_info = json.loads(f.read())

In [6]:
db: Session = next(get_db())

In [7]:
from core.databases.models.shopee import Restaurant

In [8]:
db.query(Restaurant).all()

[]

In [9]:
restaurants = [
    dict(
        id=info['restaurant_info']['id'],
                name=info['restaurant_info']['name'],
                total_order=info['restaurant_info']['total_order'],
                city_id=info['restaurant_info']['city_id'],
                restaurant_id=info['restaurant_info']['restaurant_id'],
                restaurant_url=info['restaurant_info']['restaurant_url'],
                logo_mms_img_id=info['restaurant_info']['logo_mms_img_id'],
                brand_id=info['restaurant_info']['brand_id'],
                is_open=info['restaurant_info']['is_open'],
                contract_type=info['restaurant_info']['contract_type'],
                location_url=info['restaurant_info']['location_url'],
                has_contract=info['restaurant_info']['has_contract'],
                is_quality_merchant=info['restaurant_info']['is_quality_merchant'],
                merchant_time=info['restaurant_info']['merchant_time'],
                service_type=info['restaurant_info']['service_type'],
                url_rewrite_name=info['restaurant_info']['url_rewrite_name'],
                is_foody_delivery=info['restaurant_info']['is_foody_delivery'],
                address=info['restaurant_info']['address'],
                foody_service_id=info['restaurant_info']['foody_service_id'],
                url=info['restaurant_info']['url'],
                display_order=info['restaurant_info']['display_order'],
                delivery_id=info['restaurant_info']['delivery_id'],
                is_pickup=info['restaurant_info']['is_pickup'],
                banner_mms_img_id=info['restaurant_info']['banner_mms_img_id'],
                latitude=info['restaurant_info']['position']['latitude'],
                longitude=info['restaurant_info']['position']['longitude'],
                is_verified=info['restaurant_info']['position']['is_verified'],
                district_id=info['restaurant_info']['district_id'],
                limit_distance=info['restaurant_info']['limit_distance'],
                restaurant_status=info['restaurant_info']['restaurant_status'],
                image_name=info['restaurant_info']['image_name'],
    ) for info in restaurant_info
]

In [10]:
db.bulk_insert_mappings(mapper=Restaurant, mappings=restaurants)

In [11]:
restaurant = db.query(Restaurant).first()

In [12]:
restaurant.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x122832740>,
 'city_id': 219,
 'has_contract': True,
 'address': '23 Xô Viết Nghệ Tĩnh, P. Hòa Cường Nam, Quận Hải Châu, Đà Nẵng',
 'is_pickup': False,
 'restaurant_id': 135257,
 'is_quality_merchant': False,
 'name': 'Tứ Hải Quán - Cơm Gà & Cơm Sườn',
 'banner_mms_img_id': 'vn-11134513-7r98o-lsttvkvjwkw44a',
 'district_id': 31,
 'merchant_time': 12,
 'foody_service_id': 1,
 'url_rewrite_name': 'tu-hai-quan-com-ga-com-suon',
 'logo_mms_img_id': 'vn-11134513-7r98o-lsttvlgn1py1d6',
 'service_type': 1,
 'url': 'https://shopeefood.vn/da-nang/tu-hai-quan-com-ga-com-suon',
 'latitude': 16.03270977,
 'brand_id': 1597,
 'is_foody_delivery': True,
 'display_order': 11568,
 'longitude': 108.22325934,
 'id': 12018,
 'is_open': True,
 'limit_distance': 20000,
 'delivery_id': 12018,
 'is_verified': True,
 'contract_type': 1,
 'image_name': 'foody-upload-api-foody-mobile-9-191014135205.jpg',
 'restaurant_url': 'tu-hai-quan-com-ga-com-suo

In [13]:
db.commit()